In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import cv2
import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
with open('DataSplit.pickle', 'rb') as handle:
    DataSplit = pickle.load(handle)

In [3]:
# Hyperparameters
train_batch_size = 10
val_batch_size = 10
learning_rate = 0.0001
num_epochs = 200

In [4]:
class Senz3DDataset(Dataset):
    def __init__(self, paths, labels):
        self.paths = paths
        self.labels = labels
        self.GT_labels = {'1': 5, '2': 2, '3': 3, '4': 5, '5': 0, '6': 2, 
                          '7': 1, '8': 4, '9': 3, '10': 1, '11': 1}
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        img = cv2.imread("..\\." + self.paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img.shape[1]//4, img.shape[0]//4)).astype(np.uint8)
        
        label = torch.tensor(self.GT_labels[(self.labels[idx])])
        
        return {'img': img, 'label': label}

In [6]:
img = cv2.imread("..\\." + list(DataSplit['train'].keys())[0])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float)
print(img.shape)
img = cv2.resize(img, (img.shape[1]//4, img.shape[0]//4)).astype(np.uint8)
print(img.shape)

(480, 640, 3)
(120, 160, 3)


In [7]:
# Make Datasets
TrainDataset = Senz3DDataset(list(DataSplit['train'].keys()), list(DataSplit['train'].values()))
ValDataset = Senz3DDataset(list(DataSplit['val'].keys()), list(DataSplit['val'].values()))
TestDataset = Senz3DDataset(list(DataSplit['test'].keys()), list(DataSplit['test'].values()))

In [8]:
#  Make DataLoaders
TrainDataLoader = DataLoader(TrainDataset, batch_size=train_batch_size, shuffle=True)
ValDataLoader = DataLoader(ValDataset, batch_size=val_batch_size, shuffle=True)
TestDataLoader = DataLoader(TestDataset, batch_size=1, shuffle=False)

In [9]:
import torchvision.models as models
model_name = 'resnet18'
model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(in_features=512, out_features=1000, bias=True),
    nn.ReLU(),
    nn.Linear(1000, 500),
    nn.ReLU(),
    nn.Linear(500,50),
    nn.ReLU(),
    nn.Linear(50,6),
    nn.Softmax(dim=0),    
    )

for i, param in enumerate(model.parameters()):
    if i < 60:
      print(f'Layer{i} frozen:', type(param), param.size())
      param.requires_grad = False
    else: 
      print(f'Layer{i} not frozen', type(param), param.size())
      param.requires_grad = True


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
loss_metric = nn.CrossEntropyLoss()

Layer0 frozen: <class 'torch.nn.parameter.Parameter'> torch.Size([64, 3, 7, 7])
Layer1 frozen: <class 'torch.nn.parameter.Parameter'> torch.Size([64])
Layer2 frozen: <class 'torch.nn.parameter.Parameter'> torch.Size([64])
Layer3 frozen: <class 'torch.nn.parameter.Parameter'> torch.Size([64, 64, 3, 3])
Layer4 frozen: <class 'torch.nn.parameter.Parameter'> torch.Size([64])
Layer5 frozen: <class 'torch.nn.parameter.Parameter'> torch.Size([64])
Layer6 frozen: <class 'torch.nn.parameter.Parameter'> torch.Size([64, 64, 3, 3])
Layer7 frozen: <class 'torch.nn.parameter.Parameter'> torch.Size([64])
Layer8 frozen: <class 'torch.nn.parameter.Parameter'> torch.Size([64])
Layer9 frozen: <class 'torch.nn.parameter.Parameter'> torch.Size([64, 64, 3, 3])
Layer10 frozen: <class 'torch.nn.parameter.Parameter'> torch.Size([64])
Layer11 frozen: <class 'torch.nn.parameter.Parameter'> torch.Size([64])
Layer12 frozen: <class 'torch.nn.parameter.Parameter'> torch.Size([64, 64, 3, 3])
Layer13 frozen: <class 't

In [11]:
# Initialize
epoch_train_loss = []
epoch_train_accuracy = []

epoch_val_loss = []
epoch_val_accuracy = []

epoch_test_loss = []
epoch_test_accuracy = []

best_val_accuracy = 0

for epic in range(num_epochs):
    
    # Put model in training mode
    model.train()
    
    train_correct = 0
    train_total = 0
    train_loss = 0
    
    for i, sample in tqdm.tqdm(enumerate(TrainDataLoader)):
        
        # move inputs to device
        if torch.cuda.is_available():
            x = sample['img'].permute(0, 3, 1, 2).float().to(device)
            y = sample['label'].to(device)
        
        # Loop over model, calculate loss
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_metric(output, y)
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy/loss
        _, y_hat = torch.max(output, dim=1)
        
        train_correct += torch.sum(y_hat == y)
        train_total += x.shape[0]
        train_loss += loss.item() * x.shape[0]
        
    # Append to epoch loss/accuracy
    epoch_train_loss.append(train_loss / train_total)
    epoch_train_accuracy.append(train_correct.float() / train_total)
    
    print(f'Epoch: {epic}, Train Loss: {epoch_train_loss[-1]}, Train Acc: {epoch_train_accuracy[-1]}')
    
    # Put model in validation mode
    model.eval()
    
    val_total = 0
    val_correct = 0
    val_loss = 0
    
    with torch.no_grad():
        for i, sample in enumerate(ValDataLoader):
            
            # move inputs to device
            if torch.cuda.is_available():
                x = sample['img'].permute(0, 3, 1, 2).float().to(device)
                y = sample['label'].to(device)

            output = model.forward(x)
            loss = loss_metric(output, y)
            
            # Calculate accuracy/loss
            _, y_hat = torch.max(output, dim=1)
            batch_correct = torch.sum(y_hat == y)
            val_correct += torch.sum(y_hat == y)
            val_total += x.shape[0]
            val_loss += loss.item() * x.shape[0]
            
    # Append to epoch loss/accuracy
    epoch_val_loss.append(val_loss / val_total)
    val_accuracy = val_correct.float() / val_total
    epoch_val_accuracy.append(val_accuracy)
    
    if val_accuracy > best_val_accuracy:
        print("Saved new best model.")
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), f'saved_models/{model_name}.pth')
    
    print(f'Epoch: {epic}, Val Loss: {epoch_val_loss[-1]}, Val Acc: {epoch_val_accuracy[-1]}')
    
    

97it [00:08, 12.01it/s]


Epoch: 0, Train Loss: 1.6367327076376166, Train Acc: 0.6407061815261841


0it [00:00, ?it/s]

Saved new best model.
Epoch: 0, Val Loss: 1.5706152092638632, Val Acc: 0.6187050342559814


97it [00:08, 11.19it/s]


...

Epoch: 198, Train Loss: 1.436539824382786, Train Acc: 0.9626168608665466


2it [00:00, 11.75it/s]

Epoch: 198, Val Loss: 1.4354863063894587, Val Acc: 0.9136691093444824


97it [00:08, 11.95it/s]


Epoch: 199, Train Loss: 1.4323983225867012, Train Acc: 0.9511942267417908
Epoch: 199, Val Loss: 1.428825389567039, Val Acc: 0.9136691093444824


In [1]:
plt.figure(1)
plt.plot(epoch_train_loss)
plt.plot(epoch_val_loss)
plt.title("Loss")
plt.legend(("Train", "Val"))

plt.figure(2)
plt.plot(epoch_train_accuracy)
plt.plot(epoch_val_accuracy)
plt.title("Accuracy")
plt.legend(("Train", "Val"))

NameError: name 'plt' is not defined

In [13]:
max(epoch_val_accuracy)

tensor(0.9784, device='cuda:0')